In [11]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

In [12]:
data = pd.read_csv("activity_new_data.txt", delimiter=r"\s+", header=None)
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

In [13]:

n_folds = 2
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

In [14]:
models = {
    "SVM (linear)": SVC(kernel='linear'),
    "SVM (RBF)": SVC(kernel='rbf'),
    "LDA": LDA(),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "MLP": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
}

In [15]:
results = []
for model_name, clf in models.items():
    cv_y_test = []
    cv_y_pred = []
    for train_index, test_index in kf.split(X, y):
        x_train = X[train_index, :]
        y_train = y[train_index]
        clf_cv = models[model_name]
        clf_cv.fit(x_train, y_train)
        # Test phase
        x_test = X[test_index, :]
        y_test = y[test_index]
        y_pred = clf_cv.predict(x_test)
        # Concatenate results of evaluation
        cv_y_test.append(y_test)
        cv_y_pred.append(y_pred)
        # Model performance
    print(f"\n===== {model_name} =====")
    print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))
    report_dict = classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred), output_dict=True)
    for label, metrics in report_dict.items():
        if label not in ["accuracy", "macro avg", "weighted avg"]:
            results.append({
                "Model": model_name,
                "Precision": metrics["precision"],
                "Recall": metrics["recall"],
                "F1-Score": metrics["f1-score"],
                "Support": metrics["support"]
            })
df = pd.DataFrame(results)
df.to_csv("classification_report_results.csv", index=False)
print("Resultados guardados en 'classification_report_results2.csv'")

/Users/diegovertiz/Documents/AACuarto Semestre/Modelación Inteligencia Artificial/ia_modelling/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


ValueError: Found array with 0 feature(s) (shape=(360, 0)) while a minimum of 1 is required by SVC.